This script shows basic functionalities of visdom.

To start the visdom server enter the following command in terminal:
python -m visdom.server

The visdom environment can be acessed in a browser at  http://localhost:8097


In [ ]:
#Plotting some text
import visdom
import numpy as np
#plot this text in a window
vis = visdom.Visdom()
vis.text('Hello World! Welcome to this tutorial on visdom')


In [ ]:
#plotting an image to a visdom window
import cv2

img=cv2.imread('/Users/goutham/Development/DATASETS/Patch_based__HE_sampler/val/nuc/Spot_1_HE_patch_nuclei_60.png')
img=np.transpose(img)  # the image must be in the format of CxHxW 

vis.image(
        img,
        win='old_img',
        opts=dict(
            title='H&E image',
            height=256,
            width=256,
            store_history=1
            )
        
    )


In [ ]:
#Plotting a scatter plot
X=np.random.rand(100, 2)
X=np.arange(0,10)
Y=np.random.rand(10)

Z=np.column_stack((X,Y))
# Input for a scatter plot must be a 2d or 3d matrix in format Nx2 or Nx3
vis.scatter(
        Z,
        opts=dict(
                xtickmin=0,
                xtickmax=10,
                xtickstep=1,
                ytickmin=-1,
                ytickmax=1,
                ytickstep=.2,
                legend=['Y','X'],
                xlabel=('X'),
                ylabel=('Y'),
                title=('This is a scatter plot')
               )
        )

In [ ]:
# plotting a line graph can specify Nx1 vector for x coordinates and another Nx1 vector for y coordinate.
# This graph will be appended in a later cell
A=np.arange(0,10)
B=np.zeros(10)
AB=np.column_stack((A,B))
vis.line(
        X=A,
        Y=B,
        win='old_line',
        opts=dict(
                title='line plot1 OLD',
                xlabel='x',
                ylabel='y',
                legend=['A','B'])
        )


In [ ]:
# plotting multiple lines on the same graph
A=np.random.rand(10)
B=np.random.rand(10)
AB=np.column_stack((A,B))
vis.line(
        X=np.arange(10),
        Y=AB,
        opts=dict(
                title='line plot',
                xlabel='x',
                ylabel='y',
                legend=['A','B']),
        )
        
    
    

In [ ]:
# plotting a histogram
A=np.random.rand(10)

vis.histogram(A,
              opts=dict(
                      title='This is a histogram',
                      xlabel='Variable of interest',
                      ylabel='Frequency',
                      numbins=3)
              )

In [ ]:
#updating a previous window
X=np.arange(10,20) 
Y=np.random.rand(10)
Z=np.column_stack((X,Y))
vis.line(
        X=X,
        Y=Y,
        win='old_line',
        opts=dict(
                title='line plot2',
                xlabel='xhjhj',
                ylabel='yhjhjhj',
                legend=['A','B']
                ),
                update='apend'
    
        )

Callbcks are a cool feature that allow the back end to react to events that happen in the front end. A few examples are shown below

In [ ]:
#Callbacks on visdom
def show_color_image_window(color, win=None):
    image = np.full([3, 256, 256], color, dtype=float)
    return vis.image(
        image,
        opts=dict(title='Colors', caption='Press arrows to alter color.'),
        win=win
        )

image_color = 0
callback_image_window = show_color_image_window(image_color)

def image_callback(event):
    global image_color
    if event['event_type'] == 'KeyPress':
        if event['key'] == 'ArrowRight':
            image_color = min(image_color + 0.2, 1)
        if event['key'] == 'ArrowLeft':
            image_color = max(image_color - 0.2, 0)
        show_color_image_window(image_color, callback_image_window)


        
vis.register_event_handler(image_callback, callback_image_window)


In [ ]:
txt = 'This is a notepad'
callback_text_window = vis.text(txt)

def type_callback(event):
        if event['event_type'] == 'KeyPress':
            curr_txt = event['pane_data']['content']
            if event['key'] == 'Enter':
                curr_txt += '<br>'
            elif event['key'] == 'Backspace':
                curr_txt = curr_txt[:-1]
            elif event['key'] == 'Delete':
                curr_txt = txt
            elif len(event['key']) == 1:
                curr_txt += event['key']
            vis.text(curr_txt, win=callback_text_window)


vis.register_event_handler(type_callback, callback_text_window)